### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neighbors import KDTree
import random
import matplotlib.pyplot as plt
import time
import pickle
from IPython.display import Image

### Считаем данные

Теоретические данные

In [19]:
X_test_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\X_test_array.npy")
X_train_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\X_train_array.npy")
y_test_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\y_test_array.npy")
y_train_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\y_train_array.npy")

Экспериментальные данные

In [20]:
X_exp_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_exp1\X_exp_array.npy")
X_th_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_exp1\X_th_array.npy")
y_th_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_exp1\y_th_array.npy")

In [27]:
df_ex = pd.read_csv(r'c:\Users\trrak\OneDrive\Рабочий стол\platelets\project_FC\experimental_1_degree.csv', sep=';')
df_ex = df_ex.drop('70', axis=1)

In [28]:
def weight_formula(θ):
    return (1/θ)*np.exp(-2*(np.log(θ/54))**2)

θ = np.array(df_ex.columns, dtype='double')
weight = weight_formula(θ)

In [30]:
X_train_weighted = X_train_array * weight 
X_test_weighted = X_test_array * weight 

In [33]:
X_th_weighted = X_th_array * weight
X_exp_weighted = X_exp_array * weight 

### Нахождение RMSE (минимального расстояния)

Расчет теоретических данных

In [31]:
# Создание пустого массива нужной длины для сохранения результатов
rmse_results = np.zeros(X_test_weighted.shape[0])
index_results = np.zeros(X_test_weighted.shape[0])

# Проход по каждой строке в X_test_array
for i in tqdm(range(X_test_weighted.shape[0])):
    # Считывание всех RMSE для одной строки X_test_array по X_train_array 
    # Нахождение минимального RMSE и соответствующего индекса для каждой строки X_test_array 
    rmse = np.sqrt(np.mean(np.square(X_train_weighted - X_test_weighted[i, :]), axis=1))
    min_rmse = np.min(rmse)
    min_index = np.argmin(rmse)
    # Создание два массива под RMSE и индекс
    rmse_results[i] = min_rmse
    index_results[i] = min_index

100%|██████████| 3923/3923 [03:44<00:00, 17.50it/s]


In [32]:
np.savetxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_th.txt', index_results, fmt='%i')

In [23]:
index_results

array([119668.,  73990.,  89186., ...,  45859., 101251.,  51609.])

Расчет экспериментальных данных

In [34]:
# Создание пустого массива нужной длины для сохранения результатов
rmse_results = np.zeros(X_exp_weighted.shape[0])
index_results = np.zeros(X_exp_weighted.shape[0])

# Проход по каждой строке в X_test_array
for i in tqdm(range(X_exp_weighted.shape[0])):
    # Считывание всех RMSE для одной строки X_test_array по X_train_array 
    # Нахождение минимального RMSE и соответствующего индекса для каждой строки X_test_array 
    rmse = np.sqrt(np.mean(np.square(X_th_weighted - X_exp_weighted[i, :]), axis=1))
    min_rmse = np.min(rmse)
    min_index = np.argmin(rmse)
    # Создание два массива под RMSE и индекс
    rmse_results[i] = min_rmse
    index_results[i] = min_index

100%|██████████| 17441/17441 [16:24<00:00, 17.72it/s]


In [36]:
np.savetxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_exp.txt', index_results, fmt='%i')

In [26]:
index_results

array([107345.,  81323., 167891., ..., 167967., 146545.,  86304.])

Чтение файлов, чтобы повторно не прогонять RMSE

In [15]:
index_th = np.loadtxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_th.txt')
index_exp = np.loadtxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_exp.txt')

In [16]:
index_th

array([119668.,  73990.,  89186., ...,  45859., 101251.,  51609.])

In [17]:
print(index_th.shape)
print(index_exp.shape)

(3923,)
(17441,)


Посмотрим метрику для теоретических данных по выходным параметрам

In [37]:
# Создание пустого массива для сохранения предсказанных значений
y_test_pred_array = np.zeros(y_test_array.shape)

# Проход по каждому индексу в index_results
for i, index in enumerate(index_th):
    # Извлечение соответствующего элемента из y_train_scaled и сохраняем его в y_test_pred_scaled
    y_test_pred_array[i] = y_train_array[int(index)]

# Вычисление MAE для каждой колонки
# multioutput='raw_values' возвращает массив значений MAE для каждой колонки входных данных
mae = mean_absolute_error(y_test_array, y_test_pred_array, multioutput='raw_values')
mse = mean_squared_error(y_test_array, y_test_pred_array, multioutput='raw_values')

# Вывод MAE для каждой колонки
for i, column in enumerate(['r', 'eps', 'n', 'beta']):
    print(f"MAE for {column}: {mae[i]}")
    
# Вывод MSE для каждой колонки
for i, column in enumerate(['r', 'eps', 'n', 'beta']):
    print(f"MSE for {column}: {mse[i]}")

MAE for r: 0.0432933951414731
MAE for eps: 0.02790211579913303
MAE for n: 0.003586565223043656
MAE for beta: 1.9974509304104002
MSE for r: 0.0044578123956171695
MSE for eps: 0.0015650267402501498
MSE for n: 2.970978735964952e-05
MSE for beta: 14.733622227886821
